# Prediccion del precio de las viviendas mediante regresion lineal

En este script de realizara la prediccion mediantes metodos de regresion

# Librerias

In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Datos

In [78]:
dfp=pd.read_csv(r'C:\Users\Juan Guzman\pviviendas\Data\dfanalisis')
dfp.head()

,tipo,precio,area,habitaciones,banos,garajes,balcon,trasn. publico,estrato,antiguedad,...,zonas verdes,supermercado o cc,instalacion de gas,parqueadero visitantes,colegios / universidades,parques cercanos,zona residencial,closet,municipio,tipo_propiedad
0,0,1380000000,320.00,5,6.0,2,1.0,1.0,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rionegro,casa
1,1,496122600,91.97,3,3.0,1,1.0,1.0,3.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,rionegro,apartamento
2,0,570000000,150.00,3,3.0,1,1.0,1.0,5.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,san antonio de pereira,casa
3,0,290000000,106.00,3,3.0,1,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,marinilla,casa
4,1,360002000,73.37,3,3.0,0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,rionegro,apartamento


# Procesamiento de los datos


Primero dividiremos la variable objetivo que seria el precio de las variables explicativas que serian todas las demas

In [79]:
y = dfp['precio']
X_numeric = dfp.drop(['precio','municipio','tipo_propiedad'],axis=1)
X_categoric = dfp[['municipio','tipo_propiedad']]


## Procesamiento de variables numericas

para la regresión lineal múltiple, se recomienda que las variables predictoras y la variable objetivo tengan una distribución normal y que la relación entre ellas sea lineal. Si estas condiciones no se cumplen, se pueden aplicar transformaciones a las variables para mejorar la precisión del modelo.

Debido a que las distribuciones de las variables tienen comportamiento muy variados unas de otras se optara por estandarizar la variable lo que significa centrar  la variable en $0$ y con una desviación estándar de $1$:

$$X^{\prime} = \frac{X - \mu}{\sigma}$$


In [80]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # declaramos el transformador
X_numeric_est = scaler.fit_transform(X_numeric)

## Procesamiento de variables categoricas

In [81]:
print(np.unique(X_categoric))

['apartaestudio' 'apartamento' 'casa' 'el carmen de viboral' 'el retiro'
 'finca' 'guarne' 'la ceja' 'llanogrande' 'marinilla' 'rionegro'
 'san antonio de pereira' 'santuario']


haremos el procesamiento mediante OneHotEncoder, el cual se asemeja al tratamiento de una variable dummi, es decir, para cada categoria se le asignara 1 si corresponde a esa categoria y 0 para las demas

In [82]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
X_categoric_onehot = enc.fit_transform(X_categoric)
X_categoric.shape , X_categoric_onehot.shape

((2144, 2), (2144, 13))

Podemos observar que se teniamos 13 categorias en las dos columnas

Ahora procederemos a unir los dos conjuntos de datos en la variable X

In [83]:
X = np.concatenate((X_numeric_est,X_categoric_onehot),axis=1)
dfp.shape , X.shape

((2144, 23), (2144, 33))

# Entrenamiento del modelo

Para este caso dividiremos los datos de la siguiente manera:
* 60% entrenamiento
* 20% test
* 20%validacion

Debido a que la regreison lineal no permite modificar muchos hiperparametros no entramos en la necesidad de utilizar un porcentaje de los datos para validar.
Con el fin de realizar un buen entrenamiento del algoritmo y mejorar su capacidad predictiva

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1234)
# particion de los datos de validacion en base a los datos de entrenamiento
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1234)

In [85]:
from sklearn.model_selection import train_test_split

# Primera división en train y test (80% y 20% respectivamente)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Segunda división en train y validación (60% y 20% respectivamente)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=1234)

# Imprimir las dimensiones de cada conjunto
print(f"Conjunto de entrenamiento: {X_train.shape[0]}, {y_train.shape[0]}, en porcentaje correspone a {round(((X_train.shape[0]/X.shape[0])*100),0)} del total de la muestra")
print(f"Conjunto de validación: {X_val.shape[0]}, {y_val.shape[0]}, en porcentaje correspone a {round(((X_val.shape[0]/X.shape[0])*100),0)} del total de la muestra")
print(f"Conjunto de prueba: {X_test.shape[0]}, {y_test.shape[0]}, en porcentaje correspone a {round(((X_test.shape[0]/X.shape[0])*100),0)} del total de la muestra")


Conjunto de entrenamiento: 1286, 1286, en porcentaje correspone a 60.0 del total de la muestra
Conjunto de validación: 429, 429, en porcentaje correspone a 20.0 del total de la muestra
Conjunto de prueba: 429, 429, en porcentaje correspone a 20.0 del total de la muestra


# Modelo

## Regresion lineal 

In [86]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': 'deprecated',
 'positive': False}

## Regresion Ridge

In [104]:
from sklearn.linear_model import Ridge
ridge = Ridge(random_state=42)
ridge.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': 42,
 'solver': 'auto',
 'tol': 0.001}

## Regresion Lasso

In [88]:
from sklearn.linear_model import Lasso
lasso = Lasso(random_state=42)
lasso.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': 'deprecated',
 'positive': False,
 'precompute': False,
 'random_state': 42,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

# Validacion cruzada

## Regresion lineal

In [89]:
param_grid_linear = {'fit_intercept': [True, False],
                    'normalize': [True, False],
                    'copy_X': [True, False],
                    'positive': [True, False]
}
param_grid_linear

{'fit_intercept': [True, False],
 'normalize': [True, False],
 'copy_X': [True, False],
 'positive': [True, False]}

In [90]:
from sklearn.model_selection import GridSearchCV
gridlinear = GridSearchCV(linear, param_grid_linear, cv=5) # cv son las divisiones de la validacion cruzada

### Entrenamiento del modelo lineal con GridSearchCV

In [91]:
gridlinear.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False],
                         'positive': [True, False]})

In [92]:
print(gridlinear.best_params_)
print(gridlinear.best_estimator_)

{'copy_X': True, 'fit_intercept': False, 'normalize': True, 'positive': False}
LinearRegression(fit_intercept=False, normalize=True)


## Regresion Ridge

sus parametros son:
* alpha(float):  parametro de regularicacion que controla la fuerza de la penalizacion de la norma L2. el valor predeterminado es 1.0. el alpha= 0 es lo mismo que una regreison lineal
* fit_intercept(bool): Ya sea para ajustar la intersección para este modelo. Si se establece en falso, no se utilizará intercepción en los cálculos
* normalize(bool): indica si se debe normalizar las variables de entrada
* copy_X(bool): indica si se deben copiar los datos de entrada
* max_iter(int): Número máximo de iteraciones para el solucionador de gradiente conjugado
* tol(float): Precisión de la solución
* Solver(): metodo de solucion para el problema de optimizacion. Los valores posibles son auto,svd,cholesky,lsqr o sparse_cg

In [112]:
param_grid_ridge = {
    'alpha': [0.1,1.0,10.0],
    'fit_intercept': [True,False],
    'normalize': [True,False],
    'copy_X': [True,False],
    'max_iter':np.linspace(100,1000,10),
    'tol':np.linspace(1e-4,1e-1,10),
    'solver':['auto','svd','cholesky','lsqr','sparse_cg'],
}

In [115]:
gridridge = GridSearchCV(ridge, param_grid_ridge, cv=5)

### Entrenamiento del modelo ridge con GridSearchCV

In [116]:
gridridge.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Ridge(random_state=42),
             param_grid={'alpha': [0.1, 1.0, 10.0], 'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'max_iter': array([ 100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,  900.,
       1000.]),
                         'normalize': [True, False],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg'],
                         'tol': array([0.0001, 0.0112, 0.0223, 0.0334, 0.0445, 0.0556, 0.0667, 0.0778,
       0.0889, 0.1   ])})

In [117]:
print(gridridge.best_params_)
print(gridridge.best_estimator_)

{'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'max_iter': 100.0, 'normalize': True, 'solver': 'lsqr', 'tol': 0.0001}
Ridge(alpha=0.1, max_iter=100.0, normalize=True, random_state=42, solver='lsqr',
      tol=0.0001)


## Regresion Lasso

sus parametros son:
* alpha(float):  Constante que multiplica el término L1, controlando la fuerza de regularización. alphadebe ser un flotante no negativo, es decir, en 0, inf. el alpha= 0 es lo mismo que una regreison lineal
* fit_intercept(bool): Ya sea para ajustar la intersección para este modelo. Si se establece en falso, no se utilizará intercepción en los cálculos
* normalize(bool): indica si se debe normalizar las variables de entrada
* copy_X(bool): indica si se deben copiar los datos de entrada
* max_iter(int): Número máximo de iteraciones
* tol(float): Precisión de la solución
* precompute(bool): Ya sea para usar una matriz Gram precalculada para acelerar los cálculos
* warm_start(bool): Cuando se establece en True, reutiliza la solución de la llamada anterior para que se ajuste como inicialización; de lo contrario, simplemente borra la solución anterior
* selection(ciclico, aleatorio): Si se establece en 'aleatorio', se actualiza un coeficiente aleatorio en cada iteración en lugar de recorrer las funciones de forma secuencial de forma predeterminada

In [119]:
param_grid_lasso = {
   'alpha': [0.1, 1.0, 10.0],
   'fit_intercept': [True, False],
   'normalize': [True, False],
   'copy_X': [True, False],
   'max_iter': np.linspace(100,1000,10),
   'tol': np.linspace(1e-4,1e-1,10),
   'selection': ['cyclic', 'random']
}


In [120]:
gridlasso = GridSearchCV(lasso, param_grid_lasso, cv=10)

### Entrenamiento del modelo ridge con GridSearchCV

In [121]:
gridlasso.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=Lasso(random_state=42),
             param_grid={'alpha': [0.1, 1.0, 10.0], 'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'max_iter': array([ 100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,  900.,
       1000.]),
                         'normalize': [True, False],
                         'selection': ['cyclic', 'random'],
                         'tol': array([0.0001, 0.0112, 0.0223, 0.0334, 0.0445, 0.0556, 0.0667, 0.0778,
       0.0889, 0.1   ])})

In [122]:
print(gridridge.best_params_)
print(gridridge.best_estimator_)

{'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'max_iter': 100.0, 'normalize': True, 'solver': 'lsqr', 'tol': 0.0001}
Ridge(alpha=0.1, max_iter=100.0, normalize=True, random_state=42, solver='lsqr',
      tol=0.0001)


# Evaluacion del modelo